In [31]:
#สร้างและบันทึกโมเดล Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [32]:
#โหลดโมเดล Word2Vec ที่บันทึกไว้ ขึ้นมาใช้งาน และทำการ train ต่อ
model = Word2Vec.load("word2vec.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)
(0, 2)

(0, 2)

In [33]:
#หาค่า similarlity
vector = model.wv['computer']  # get numpy vector of a word
sims = model.wv.most_similar('computer', topn=10)  # get other similar words

In [34]:
#โหลดกลับมาใช้งานแบบ memory-mapped
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = wv['computer']  # Get numpy vector of a word

In [35]:
#ดึงเส้นทางไฟล์ที่เก็บโมเดลในรูปแบบข้อความและไบนารี่
from gensim.test.utils import datapath
# Load a word2vec model stored in the C *text* format.
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)
# Load a word2vec model stored in the C *binary* format.
wv_from_bin = KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)

In [36]:
#ดึง word vectors เก็บในตัวแปร
word_vectors = model.wv
del model

In [37]:
from gensim.models import Phrases

# Train a bigram detector.
bigram_transformer = Phrases(common_texts)

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
model = Word2Vec(bigram_transformer[common_texts], min_count=1)

In [38]:
import requests
from bs4 import BeautifulSoup

# URL ของหน้าเว็บ
url = "https://clincalc.com/DrugStats/Top300Drugs.aspx"

# ส่งคำขอ GET
response = requests.get(url)

# แปลงเนื้อหาด้วย BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# หาและดึงชื่อยา (ปรับการเลือกตามโครงสร้างของ HTML)
drug_names = soup.find_all('td', class_='DrugNameClass')  # ปรับ class_ ตามที่พบ

# แสดงชื่อยา
for drug in drug_names:
    print(drug.get_text())


In [39]:
#สร้างฟังก์ชั่นนำ dataframe ข้อมูลเข้าโมเดล คำนวณ Cosine Similarity
import gensim.downloader
import numpy as np
from scipy.spatial.distance import cosine
import csv

def find_similar_herbs(med_data, herb_data, model):
    """Finds the most similar herbs for each medication based on a word embedding model.

    Args:
        med_data: A list of medication names.
        herb_data: A list of herb names.
        model: A word embedding model.

    Returns:
        A list of tuples, where each tuple contains the medication, herb, and their similarity score.
    """
    results = []

    for med in med_data:
        try:
            # Get the vector of the medication
            med_vector = model.get_vector(med)
        except KeyError:
            # If the medication is not in the model's vocabulary, skip it
            continue

        for herb in herb_data:
            try:
                # Get the vector of the herb
                herb_vector = model.get_vector(herb)
                # Compute cosine similarity
                similarity = 1 - cosine(med_vector, herb_vector)
                results.append((med, herb, similarity))
            except KeyError:
                # If the herb is not in the model's vocabulary, skip it
                continue

    # Sort the results by similarity (descending order)
    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return sorted_results

# Load the model
glove_vectors = gensim.downloader.load('glove-twitter-25')


[==================================================] 100.0% 104.8/104.8MB downloaded


In [40]:
#web scrpping ดึงข้อมูลยา
from bs4 import BeautifulSoup
import requests

response = requests.get('https://clincalc.com/DrugStats/Top300Drugs.aspx').text
soup = BeautifulSoup(response, 'html.parser')
df = pd.DataFrame(columns=['Rank', 'Drug_Name', 'Total_Prescriptions_(2022)', 'Total_Patients_(2022)', 'Annual_Change'])
table = soup.find('table')

rows = []
for row in table.find_all('tr'):
    columns = row.find_all('td')
    if columns:
        rank = columns[0].text.strip()
        drug = columns[1].text.strip()
        prescriptions = columns[2].text.strip()
        totalpatients = columns[3].text.strip()
        annual = columns[4].text.strip()

        rows.append({'Rank': rank, 'Drug_Name': drug, 'Total_Prescriptions_(2022)': prescriptions, 'Total_Patients_(2022)': totalpatients, 'Annual_Change': annual})

df = pd.DataFrame(rows)
df

,Rank,Drug_Name,Total_Prescriptions_(2022),Total_Patients_(2022),Annual_Change
0,1,Atorvastatin,"109,582,746","27,935,702",0
1,2,Metformin,"86,747,907","19,536,027",0
2,3,Lisinopril,"82,513,967","20,314,304",1
3,4,Levothyroxine,"82,431,914","18,130,331",1
4,5,Amlodipine,"70,766,211","17,789,649",0
...,...,...,...,...,...
295,296,Neomycin; Polymyxin B; Hydrocortisone,"418,234","364,330",8
296,297,Loperamide,"410,958","128,701",10
297,298,Azelate,"395,502","202,965",14
298,299,Azelastine; Fluticasone,"394,381","106,697",21


In [ ]:
med_data1 = df['Drug_Name'].tolist()
med_data = [x.lower() if isinstance(x, str) else x for x in med_data1]
med_data

In [56]:
#ใส่ list ข้อมูลที่จะเข้า dataframe สร้าง list ผลลัพธ์
# Define medication data

# Define herb data
herb_data = ['echinacea', 'elderberry', 'turmeric', 'pomegranate', 'oregano', 'fenugreek', 'kudzu', 'flaxseed', 'cranberry', 'moringa', 'coriander', 'rosehip', 'thyme', 'parsley', 'tulsi', 'chamomile', 'elderflower', 'calendula', 'yarrow', 'lemongrass', 'ginseng', 'saffron', 'blueberry', 'cinnamon', 'hops', 'peppermint', 'artichoke', 'rosemary', 'mallow', 'garlic', 'nettle', 'soursop', 'jojoba', 'lavender', 'hibiscus', 'hawthorn', 'arjuna', 'amla', 'blackberry', 'guarana', 'ginger', 'horsetail', 'sage']
# Find the most similar herbs for each medication
similar_herbs = find_similar_herbs(med_data, herb_data, glove_vectors)
# Print the results
for med, herb, similarity in similar_herbs:
    print(f"Target med: {med}, Similar Herb: {herb}, Similarity: {similarity:.2f}")


Target med: naproxen, Similar Herb: echinacea, Similarity: 0.81
Target med: naproxen, Similar Herb: elderberry, Similarity: 0.81
Target med: calcium, Similar Herb: turmeric, Similarity: 0.81
Target med: calcium, Similar Herb: echinacea, Similarity: 0.80
Target med: oxycodone, Similar Herb: elderberry, Similarity: 0.80
Target med: clonazepam, Similar Herb: echinacea, Similarity: 0.80
Target med: diclofenac, Similar Herb: echinacea, Similarity: 0.80
Target med: naproxen, Similar Herb: turmeric, Similarity: 0.78
Target med: promethazine, Similar Herb: pomegranate, Similarity: 0.78
Target med: alprazolam, Similar Herb: oregano, Similarity: 0.78
Target med: metformin, Similar Herb: elderberry, Similarity: 0.78
Target med: oxycodone, Similar Herb: echinacea, Similarity: 0.78
Target med: diazepam, Similar Herb: oregano, Similarity: 0.77
Target med: naproxen, Similar Herb: fenugreek, Similarity: 0.77
Target med: clonazepam, Similar Herb: elderberry, Similarity: 0.77
Target med: diclofenac, Sim

In [57]:
# export data เป็นไฟล์ excel
import pandas as pd

# สร้าง DataFrame จากข้อมูล
data = []
for med, herb, similarity in similar_herbs:
    data.append({"Target med": med, "Similar Herb": herb, "Similarity": round(similarity, 2)})

df = pd.DataFrame(data)

# บันทึกลงไฟล์ Excel
output_file = "similar_herbtry.xlsx"
df.to_excel(output_file, index=False)

print(f"Results saved to {output_file}")


Results saved to similar_herbtry.xlsx
